In [28]:
import numpy as np
import copy

f1 = open('bioinfo2017/insulin_pep.uniprot.fa','r')

#Make prp_seq data
seq_data = dict()
for line in f1:
    if(line.startswith('>')):
        seq_h = line.strip().split(' ')[0].split('|')[2]
        seq_data[seq_h] = ''
    else:
        seq_data[seq_h] += line.strip()
f1.close()
        
#Find human pro_seq        
for key,value in seq_data.items():
    if key == 'INS_HUMAN':
        main_seq_key = key
        main_seq_value = value
        del seq_data[key]
        
#Make array for dynamic programming(row = Human, col = Other)
array_data = dict()
tmp = 0
for key,value in seq_data.items():        
    array_data[key] = np.zeros([len(main_seq_value)+1, len(value)+1])
    
#First row, col is -6*i
for key in array_data.keys():
    for row in range(0,array_data[key].shape[0]):
        array_data[key][row][0] += -6*row
    for col in range(0,array_data[key].shape[1]):
        array_data[key][0][col] += -6*col
        
#Make trace dict for new_pro_seq(with gap) & calculate and fill the cell
trace_data = copy.deepcopy(array_data)
for key in array_data.keys():
    for row in range(1,array_data[key].shape[0]):
        for col in range(1,array_data[key].shape[1]):
            if (main_seq_value[row-1] == seq_data[key][col-1]):
                match_score = 5
            else:
                match_score = -2
            max_value = max(array_data[key][row-1][col-1] + match_score ,array_data[key][row][col-1] -6,array_data[key][row-1][col]-6)
            array_data[key][row][col] += max_value
            
            if (max_value == array_data[key][row-1][col-1] + match_score ):
                trace_data[key][row][col] += 1 #diagonal
            elif (max_value == array_data[key][row][col-1] -6):
                trace_data[key][row][col] += 2 #horizontal
            else:
                trace_data[key][row][col] += 3 #vertical

#Make new_pro_seq from trace dict
new_seq = copy.deepcopy(seq_data)
for key in trace_data.keys():
    new_seq[key] = ''
    col = trace_data[key].shape[1]-2
    for row in range(trace_data[key].shape[0]-2,0,-1):
        if (trace_data[key][row][col] == 1):
            new_seq[key] += seq_data[key][col]
            col -= 1
        elif (trace_data[key][row][col] == 3):
            new_seq[key] += '-'
for key in trace_data.keys():
    new_seq[key] += 'M'           
            
            
#Print out
print 'name\tscore\n' ,'INS_HUMAN\n', main_seq_value
for key in array_data.keys():
    print key,'\t',array_data[key][-1][-1]
    print new_seq[key][::-1]

name	score
INS_HUMAN
MALWMRLLPLLALLALWGPDPAAAFVNQHLCGSHLVEALYLVCGERGFFYTPKTRREAEDLQVGQVELGGGPGAGSLQPLALEGSLQKRGIVEQCCTSICSLYQLENYCN
INS_DANRE 	103.0
MAVWLQAGALLVLLV-VSSVSTNPGTPQHLCGSHLVDALYLVCGPTGFFYNPKRDVEPLLGFLPPKSAQETEVADFAFKDHAEL-IRKRGIVEQCCHKPCSIFELQNYCN
INS_CHICK 	260.0
MALWIRSLPLLALLVFSGPGTSYAAANQHLCGSHLVEALYLVCGERGFFYSPKARRDVEQPLVS-SPLR-GE-AGVLPFQQEEYEKVKRGIVEQCCHNTCSLYQLENYCN
INS1_XENLA 	207.0
MALWMQCLPLVLVLFFSTPN-TEALVNQHLCGSHLVEALYLVCGDRGFFYYPKVKRDMEQALVSGPQDNELDGM-QLQPQ--EYQKMKRGIVEQCCHSTCSLFQLESYCN
INS1_MOUSE 	369.0
MALLVHFLPLLALLALWEPKPTQAFVKQHLCGPHLVEALYLVCGERGFFYTPKSRREVEDPQVEQLELGGSPG--DLQTLALEVARQKRGIVDQCCTSICSLYQLENYCN
INS_BOVIN 	371.0
MALWTRLRPLLALLALWPPPPARAFVNQHLCGSHLVEALYLVCGERGFFYTPKARREVEGPQVGALELAGGPGAGG-----LEGPPQKRGIVEQCCASVCSLYQLENYCN
INS_PIG 	432.0
MALWTRLLPLLALLALWAPAPAQAFVNQHLCGSHLVEALYLVCGERGFFYTPKARREAENPQAGAVELGGGLGG-L-QALALEGPPQKRGIVEQCCTSICSLYQLENYCN
